In [1]:
import gym
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import numpy as np
from memory_module import replayBuffer

In [2]:
def  Neural_Network():
    state_input=tf.keras.layers.Input((4,),name='state_input')
    action_input=tf.keras.layers.Input((2,),name='action_input')

    net=tf.keras.layers.Dense(256,'relu')(state_input)
    net=tf.keras.layers.Dense(256,'relu')(net)
    net=tf.keras.layers.Dense(2)(net)

    Q_vals=tf.multiply(net,action_input,name='masking')

    model=tf.keras.Model(inputs=[state_input,action_input],outputs=[Q_vals],name='DQN')
    model.compile(loss='mse',optimizer=tf.keras.optimizers.RMSprop(lr=0.00025, rho=0.95, epsilon=0.01))
    return model

In [3]:
dqn=Neural_Network()
fixed_dqn=Neural_Network()

W0209 23:36:05.534805 140046186432320 deprecation.py:506] From /home/abhijit/.local/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [4]:
def hard_copy(source,target):
    target.set_weights(source.get_weights())

In [5]:

def train_on_batch(s,a,r,s_,nd):
    batch_size=s.shape[0]
    q_targets=r+0.99*np.multiply(nd,np.max(fixed_dqn.predict([s_,np.ones((batch_size,2))]),axis=1))
    q_target_formated=np.multiply(q_targets.reshape(-1,1),tf.keras.utils.to_categorical(a,2))
    
    dqn.train_on_batch([s,tf.keras.utils.to_categorical(a,2)],q_target_formated)

In [6]:
def get_episilon(step,steady_episilon=0.01,steady_step=100000):
    if step>steady_step:
        return steady_episilon
    else:
        m=(steady_episilon-1)/steady_step
        return m*step+1

In [7]:
def getAction(s,episilon):
    if np.random.random()<episilon:
        return np.random.randint(2)
    else:
        optimal_action=np.argmax(dqn.predict([s.reshape(1,-1),np.ones((1,2))]),axis=1)[0]
        return optimal_action

In [8]:
env=gym.make('CartPole-v0')
env._max_episode_steps=200
memory=replayBuffer(100000)

In [9]:
curr_state=env.reset()
for _ in range(50000):
    act=env.action_space.sample()
    next_state,reward,done,info=env.step(act)
    memory.push(curr_state,act,reward,next_state,not done)
    
    if done:
        curr_state=env.reset()
    else:
        curr_state=next_state

In [ ]:
episode_reward=0
curr_state=env.reset()

for step in range(1000000):
    act=getAction(curr_state,get_episilon(step,steady_episilon=0.01, steady_step=100000))
    next_state,reward,done,info=env.step(act)
    episode_reward+=reward
    
    memory.push(curr_state,act,reward,next_state,not done)
    
    if done:
        curr_state=env.reset()
        print('On step {} episode reward {}'.format(step,episode_reward))
        episode_reward=0
    else:
        curr_state=next_state
        
    if step%10000==0:
        hard_copy(dqn,fixed_dqn)
        
    if step%4==0:
        s,a,r,s_,nd=memory.sample(32)
        train_on_batch(s,a,r,s_,nd)


In [11]:
for episode in range(10):
    episode_reward=0
    curr_state=env.reset()
    done=False
    while not done:
        env.render()
        act=getAction(curr_state,0.01)
        next_state,reward,done,info=env.step(act)
        episode_reward+=reward
        curr_state=next_state
    print("On episode {} reward {}".format(episode,episode_reward))
env.close()

On episode 0 reward 200.0
On episode 1 reward 200.0
On episode 2 reward 200.0
On episode 3 reward 200.0
On episode 4 reward 200.0
On episode 5 reward 200.0
On episode 6 reward 200.0
On episode 7 reward 200.0
On episode 8 reward 200.0
On episode 9 reward 200.0
